In [ ]:
import pandas as pd
import sqlalchemy
from matplotlib import pyplot as plt
import datetime
import numpy as np
import seaborn as sns
import pandas as pd

pd.options.mode.chained_assignment = None


In [ ]:
db = sqlalchemy.engine.url.URL(drivername='mysql',
                            host='127.0.0.1',
                            database='sherman_1',
                            query={'read_default_file': '~/.my.cnf', 'charset':'utf8'})
engine = sqlalchemy.create_engine(db)

In [ ]:
df = pd.read_sql('''
    SELECT * FROM sherman_1.calls;
''' ,engine)

In [ ]:
# merge sensor info for the same participant
df_id = pd.read_csv('/home/memari/jhu_centering/tables/id.csv')
# second device_id
l = [17,18,61,20,22,37]
l2 = set()
for index in l:
    l2.add(df_id.loc[index,'device_id2'])
# create a dict of 'device_id2':'device_id1'
my_dict = {}
for index, row in df_id.iterrows():
    if row['device_id2'] in l2:
        my_dict[row['device_id2']] = row['device_id1']
# replace
for index, row in df.iterrows():
    if row['device_id'] in my_dict:
        df.loc[index,'device_id'] = my_dict[row['device_id']]

In [ ]:
def get_incoming_outgoing_missed(df):
    uniq_device_id = list(set(df['device_id'].tolist()))
    df_output = pd.DataFrame()
    df_incoming = pd.DataFrame()
    df_outgoing = pd.DataFrame()
    df_missed = pd.DataFrame()
    for device_id in uniq_device_id:
        df_target = df[df['device_id']==device_id]
        df_target = df_target.sort_values(by = 'timestamp', ascending = False)
        
        df_output_temp = df_target[['device_id','timestamp']]
        
        past_type = df_target['call_type'].tolist()
        past_type = past_type[1:]+[None]
        future_type = df_target['call_type'].tolist()
        future_type = [None]+future_type[:-1]
        duration = df_target['call_duration'].tolist()
        duration = [None]+duration[:-1]
        
        df_target['past_type'] = past_type
        df_target['future_type'] = future_type
        df_target['duration'] = duration
        
        

        #incoming
        df_incoming_temp = df_target[(df_target['past_type']==1)&
                                   (df_target['call_type']==2)&
                                   (df_target['future_type']==4)]
        
        df_incoming_temp['incoming_time'] = df_incoming_temp['duration']
        df_incoming_temp = df_incoming_temp[['device_id','timestamp','incoming_time']]
        
        
        #outgoing
        df_outgoing_temp = df_target[(df_target['past_type']==3)&
                                   (df_target['call_type']==2)&
                                   (df_target['future_type']==4)]
        
        df_outgoing_temp['outgoing_time'] = df_outgoing_temp['duration']
        df_outgoing_temp = df_outgoing_temp[['device_id','timestamp','outgoing_time']]
        
        
        #missing
        df_missing_temp = df_target[(df_target['call_type']==4)&
                                   (df_target['past_type']!=2)]
        
        df_missing_temp['missing_time'] = df_missing_temp['duration']
        df_missing_temp = df_missing_temp[['device_id','timestamp','missing_time']]
        
        df_output_temp = df_output_temp.merge(df_incoming_temp, on = ['device_id','timestamp'], how = 'left')
        df_output_temp = df_output_temp.fillna(0)
        
        df_output_temp = df_output_temp.merge(df_outgoing_temp, on = ['device_id','timestamp'], how = 'left')
        df_output_temp = df_output_temp.fillna(0)
        
        df_output_temp = df_output_temp.merge(df_missing_temp, on = ['device_id','timestamp'], how = 'left')
        df_output_temp = df_output_temp.fillna(0)
        
        if df_output_temp.shape[0]>0:
            df_output = pd.concat([df_output, df_output_temp], ignore_index=True)
    
    return df_output

In [ ]:
df_less = get_incoming_outgoing_missed(df)
df_less['date'] = pd.to_datetime(df_less['timestamp'],unit='ms').dt.date

In [ ]:
df_less = df_less[['device_id','date','incoming_time','outgoing_time','missing_time']]
df_less['incoming_freq'] = df_less['incoming_time'].apply(lambda x: int(x>0))
df_less['outgoing_freq'] = df_less['outgoing_time'].apply(lambda x: int(x>0))
df_less['missing_freq'] = df_less['missing_time'].apply(lambda x: int(x>0))

In [ ]:
df_output = df_less.groupby(['device_id','date']).agg(incoming_time = ('incoming_time', np.sum),
                                                      incoming_freq = ('incoming_freq',np.sum),
                                                      outgoing_time = ('outgoing_time', np.sum),
                                                      outgoing_freq = ('outgoing_freq',np.sum),
                                                      missing_time = ('missing_time', np.sum),
                                                      missing_freq = ('missing_freq',np.sum),
                                                     )
df_output = df_output.reset_index()
df_output['incoming_freq'] = df_output['incoming_freq'].apply(lambda x: int(x))
df_output['outgoing_freq'] = df_output['outgoing_freq'].apply(lambda x: int(x))
df_output['missing_freq'] = df_output['missing_freq'].apply(lambda x: int(x))
df_output

In [ ]:
df_output.to_csv("/home/memari/jhu_centering/tables/calls_cleaned.csv", index = False)

The experiments have been started on 11/11/2022.
We need to collect sensor information for each paritipant duraing days [1-14] and [15-28].
Each participant would start the experminext in a diffrenet day.

In [ ]:
# find the first day
def find_first_day(device_id, df_cleaned):
    df_target = df_cleaned[df_cleaned['device_id']==device_id]
    return min(df_target['date'])

In [ ]:
def get_1_14(df):
    first_day = {device_id:find_first_day(device_id, df) 
                 for device_id in set(df['device_id'].tolist())
                if find_first_day(device_id, df)>=datetime.date(year=2022,month=11,day=11)}
    df = df[df['device_id'].isin(first_day)]
    df['day'] = df.apply(lambda x: 1+(x['date']-first_day[x['device_id']]).days, axis = 1)
    df_1_14 = df[df['day']<=14]
    return df_1_14

In [ ]:
# find the first day
def find_day_15(device_id, df_cleaned):
    df_target = df_cleaned[df_cleaned['device_id']==device_id]
    return min(df_target['date'])

In [ ]:
def get_15_28(df):
    first_day = {device_id:find_first_day(device_id, df) 
                 for device_id in set(df['device_id'].tolist())
                if find_first_day(device_id, df)>=datetime.date(year=2022,month=11,day=11)}
    df = df[df['device_id'].isin(first_day)]
    df['day'] = df.apply(lambda x: 1+(x['date']-first_day[x['device_id']]).days, axis = 1)
    df_1_14 = df[df['day']<=14]

    return df_1_14,df_15_28

In [ ]:
df_1_14, df_15_28= halve(df_output)

In [ ]:
grouped_1_14 = df_1_14.groupby('date')
grouped_15_28 = df_1_14.groupby('date')

In [ ]:
grouped_1_14['incoming_time'].mean().plot(kind='bar')
plt.title('Call Incoming Call day [1-14]')
plt.show()

In [ ]:
grouped_15_28['incoming_time'].mean().plot(kind='bar')
plt.title('Call Incoming Call day [15-28]')
plt.show()